# Building a Spam Filter with Naive Bayes

The goal of this project is to "teach" computer how to classify messages. In this projet we will use the multinomial Naive Bayes algorithm along with a dataset of 5572 SMS messages that are already classified by humans. 

The dataset was put together by Tiago A. Almeida and José María Gómez Hidalgo, and it can be downloaded from the [The UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/datasets/sms+spam+collection). 

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
%matplotlib inline

In [2]:
data = pd.read_csv('SMSSpamCollection', sep = '\t', header = None, names = ['Label', 'SMS'])

In [3]:
data.head()

,Label,SMS
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [4]:
data.shape

(5572, 2)

In [5]:
data['Label'].value_counts(normalize = True)

ham     0.865937
spam    0.134063
Name: Label, dtype: float64

Above, we can see that about 87% of the messages are ham, and the remaining 13% are spam. This sample looks representative, since in practice most messages that people receive are ham.

## Training and Test Data

We're now going to split our dataset into a training and a test set, where the training set accounts for 80% of the data, and the test set for the remaining 20%.

In [6]:
data_randomized = data.sample(frac = 1, random_state = 1)

train_set_index = round(len(data_randomized)*0.8)

train_set = data_randomized[:train_set_index].reset_index(drop = True)
test_set = data_randomized[train_set_index:].reset_index(drop=True)

print(train_set.shape)
print(test_set.shape)

(4458, 2)
(1114, 2)


We'll now analyze the percentage of spam and ham messages in the training and test sets. We expect the percentages to be close to what we have in the full dataset, where about 87% of the messages are ham, and the remaining 13% are spam.

In [7]:
train_set['Label'].value_counts(normalize = True)

ham     0.86541
spam    0.13459
Name: Label, dtype: float64

In [8]:
test_set['Label'].value_counts(normalize = True)

ham     0.868043
spam    0.131957
Name: Label, dtype: float64

## Cleaning the Data

To calculate all the probabilities required by the algorithm, we'll first need to perform a bit of data cleaning to bring the data in a format that will allow us to extract easily all the information we need.

We'll begin with removing all the punctuation and bringing every letter to lower case.

In [9]:
train_set.head()

,Label,SMS
0,ham,"Yep, by the pretty sculpture"
1,ham,"Yes, princess. Are you going to make me moan?"
2,ham,Welp apparently he retired
3,ham,Havent.
4,ham,I forgot 2 ask ü all smth.. There's a card on ...


In [10]:
train_set['SMS'] = train_set['SMS'].str.replace('\W', ' ')
train_set['SMS'] = train_set['SMS'].str.lower()
train_set.head()

,Label,SMS
0,ham,yep by the pretty sculpture
1,ham,yes princess are you going to make me moan
2,ham,welp apparently he retired
3,ham,havent
4,ham,i forgot 2 ask ü all smth there s a card on ...


In [11]:
train_set['SMS'] = train_set['SMS'].str.split()

In [12]:
train_set.head()

,Label,SMS
0,ham,"[yep, by, the, pretty, sculpture]"
1,ham,"[yes, princess, are, you, going, to, make, me,..."
2,ham,"[welp, apparently, he, retired]"
3,ham,[havent]
4,ham,"[i, forgot, 2, ask, ü, all, smth, there, s, a,..."


Now move to creating the vocabulary, which in this context means a list with all the unique words in our training set.

In [15]:
vocabulary = []

for row in train_set['SMS']:
        for word in row:
            vocabulary.append(word)
    

In [17]:
print(vocabulary)

['yep', 'by', 'the', 'pretty', 'sculpture', 'yes', 'princess', 'are', 'you', 'going', 'to', 'make', 'me', 'moan', 'welp', 'apparently', 'he', 'retired', 'havent', 'i', 'forgot', '2', 'ask', 'ü', 'all', 'smth', 'there', 's', 'a', 'card', 'on', 'da', 'present', 'lei', 'how', 'ü', 'all', 'want', '2', 'write', 'smth', 'or', 'sign', 'on', 'it', 'ok', 'i', 'thk', 'i', 'got', 'it', 'then', 'u', 'wan', 'me', '2', 'come', 'now', 'or', 'wat', 'i', 'want', 'kfc', 'its', 'tuesday', 'only', 'buy', '2', 'meals', 'only', '2', 'no', 'gravy', 'only', '2', 'mark', '2', 'no', 'dear', 'i', 'was', 'sleeping', 'p', 'ok', 'pa', 'nothing', 'problem', 'ill', 'be', 'there', 'on', 'lt', 'gt', 'ok', 'my', 'uncles', 'in', 'atlanta', 'wish', 'you', 'guys', 'a', 'great', 'semester', 'my', 'phone', 'ok', 'which', 'your', 'another', 'number', 'the', 'greatest', 'test', 'of', 'courage', 'on', 'earth', 'is', 'to', 'bear', 'defeat', 'without', 'losing', 'heart', 'gn', 'tc', 'dai', 'what', 'this', 'da', 'can', 'i', 'send'

In [18]:
vocabulary = set(vocabulary)

In [19]:
vocabulary

{'plenty',
 'stomach',
 'urmom',
 'sec',
 'rain',
 '2004',
 'push',
 'march',
 'sorry',
 '28',
 'offense',
 'drink',
 'coveragd',
 'grand',
 '09065069154',
 'subscribe',
 'deliver',
 'subscribe6gbp',
 'exterminator',
 'textpod',
 'b4',
 'frontierville',
 'psychologist',
 '2mro',
 'gave',
 'newport',
 'axis',
 'attraction',
 'tonight',
 'head',
 'eightish',
 'z',
 'ntimate',
 'chit',
 '80182',
 'yelow',
 'planettalkinstant',
 'shudvetold',
 'ext',
 'with',
 'surely',
 '09061701444',
 'club4mobiles',
 'neva',
 'tot',
 'double',
 'difference',
 'bill',
 'tel',
 'avent',
 'never',
 'advise',
 'expects',
 'lionm',
 '4brekkie',
 'ge',
 '08001950382',
 'faith',
 'ee',
 'heaven',
 'pobox84',
 'upgrading',
 '861',
 'roommate',
 'bbdeluxe',
 'porn',
 'cos',
 'pthis',
 'especially',
 'offers',
 'heading',
 'teach',
 'congratulations',
 'site',
 'jenxxx',
 'hurt',
 'dload',
 'com1win150ppmx3age16subscription',
 'gibe',
 'anything',
 '373',
 'freaked',
 'bridgwater',
 '1405',
 'champlaxigating',
 '

In [20]:
vocabulary = list(vocabulary)

In [21]:
print(vocabulary)

['plenty', 'stomach', 'urmom', 'sec', 'rain', '2004', 'push', 'march', 'sorry', '28', 'offense', 'drink', 'coveragd', 'grand', '09065069154', 'subscribe', 'deliver', 'subscribe6gbp', 'exterminator', 'textpod', 'b4', 'frontierville', 'psychologist', '2mro', 'gave', 'newport', 'axis', 'attraction', 'tonight', 'head', 'eightish', 'z', 'ntimate', 'chit', '80182', 'yelow', 'planettalkinstant', 'shudvetold', 'ext', 'with', 'surely', '09061701444', 'club4mobiles', 'neva', 'tot', 'double', 'difference', 'bill', 'tel', 'avent', 'never', 'advise', 'expects', 'lionm', '4brekkie', 'ge', '08001950382', 'faith', 'ee', 'heaven', 'pobox84', 'upgrading', '861', 'roommate', 'bbdeluxe', 'porn', 'cos', 'pthis', 'especially', 'offers', 'heading', 'teach', 'congratulations', 'site', 'jenxxx', 'hurt', 'dload', 'com1win150ppmx3age16subscription', 'gibe', 'anything', '373', 'freaked', 'bridgwater', '1405', 'champlaxigating', 'eerie', 'slow', 'pull', 'tightly', 'swiss', 'leads', 'moments', 'tt', 'rakhesh', 'usu

In [22]:
len(vocabulary)

7783

Next we are going to use the vocabulary we just created to make the data transformation we want.

In [23]:
word_counts_per_sms = {unique_word:[0] * len(train_set['SMS']) for unique_word in vocabulary}

for index, sms in enumerate(train_set['SMS']):
    for word in sms:
        word_counts_per_sms[word][index] +=1

In [24]:
df = pd.DataFrame(word_counts_per_sms)

In [25]:
df.head()

,0,00,000,000pes,008704050406,0089,01223585334,02,0207,02072069400,...,zindgi,zoe,zogtorius,zouk,zyada,é,ú1,ü,〨ud,鈥
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2,0,0


In [26]:
training_set = pd.concat([train_set, df], axis = 1)
training_set.head()

,Label,SMS,0,00,000,000pes,008704050406,0089,01223585334,02,...,zindgi,zoe,zogtorius,zouk,zyada,é,ú1,ü,〨ud,鈥
0,ham,"[yep, by, the, pretty, sculpture]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,ham,"[yes, princess, are, you, going, to, make, me,...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,ham,"[welp, apparently, he, retired]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,ham,[havent],0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,ham,"[i, forgot, 2, ask, ü, all, smth, there, s, a,...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2,0,0


In order to create a Naive Bayes algorithm we need to split dataset into spam messages and ham messages. Next we have to calculate P(Spam), P(Ham), Nspam, NHam, NVocabulary to be able to compute P(wi|Spam) and P(wi|Ham). 

In [27]:
spam_mess = training_set[training_set['Label']=='spam']
ham_mess = training_set[training_set['Label']=='ham']

In [28]:
spam_mess.head()

,Label,SMS,0,00,000,000pes,008704050406,0089,01223585334,02,...,zindgi,zoe,zogtorius,zouk,zyada,é,ú1,ü,〨ud,鈥
16,spam,"[freemsg, why, haven, t, you, replied, to, my,...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
18,spam,"[congrats, 2, mobile, 3g, videophones, r, your...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
56,spam,"[free, message, activate, your, 500, free, tex...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
60,spam,"[call, from, 08702490080, tells, u, 2, call, 0...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
61,spam,"[someone, has, conacted, our, dating, service,...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [29]:
p_spam = len(spam_mess)/len(training_set)
p_ham = len(ham_mess)/len(training_set)

In [32]:
N_spam = spam_mess['SMS'].apply(len)
N_spam = N_spam.sum()
N_ham = ham_mess['SMS'].apply(len)
N_ham = N_ham.sum()

In [31]:
N_vocab = len(vocabulary)

In [33]:
alpha = 1 # Laplace smoothing 

## Calculating Parameters

Now that we have the constant terms calculated above, we can move on with calculating the parameters $P(w_i|Spam)$ and $P(w_i|Ham)$. Each parameter will thus be a conditional probability value associated with each word in the vocabulary.

In [44]:
# Initiate parameters
parameters_spam = {unique_word:0 for unique_word in vocabulary}
parameters_ham = {unique_word:0 for unique_word in vocabulary}

# Calculate parameters
for word in vocabulary:
    n_word_given_spam = spam_mess[word].sum()   # spam_messages already defined in a cell above
    p_word_given_spam = (n_word_given_spam + alpha) / (N_spam + alpha*N_vocab)
    parameters_spam[word] = p_word_given_spam
    
    n_word_given_ham = ham_mess[word].sum()   # ham_messages already defined in a cell above
    p_word_given_ham = (n_word_given_ham + alpha) / (N_ham + alpha*N_vocab)
    parameters_ham[word] = p_word_given_ham

## Classifying A New Messages

Now that we have all our parameters calculated, we can start creating the spam filter. The spam filter can be understood as a function that:

Takes in as input a new message (w1, w2, ..., wn).
Calculates P(Spam|w1, w2, ..., wn) and P(Ham|w1, w2, ..., wn).
Compares the values of P(Spam|w1, w2, ..., wn) and P(Ham|w1, w2, ..., wn), and:

- If P(Ham|w1, w2, ..., wn) > P(Spam|w1, w2, ..., wn), then the message is classified as ham.
- If P(Ham|w1, w2, ..., wn) < P(Spam|w1, w2, ..., wn), then the message is classified as spam.
- If P(Ham|w1, w2, ..., wn) = P(Spam|w1, w2, ..., wn), then the algorithm may request human help.

In [45]:
import re

def classify(message):
    '''
    message: a string
    '''
    
    message = re.sub('\W', ' ', message)
    message = message.lower().split()
    
    p_spam_given_message = p_spam
    p_ham_given_message = p_ham

    for word in message:
        if word in parameters_spam:
            p_spam_given_message *= parameters_spam[word]
            
        if word in parameters_ham:
            p_ham_given_message *= parameters_ham[word]
            
    print('P(Spam|message):', p_spam_given_message)
    print('P(Ham|message):', p_ham_given_message)
    
    if p_ham_given_message > p_spam_given_message:
        print('Label: Ham')
    elif p_ham_given_message < p_spam_given_message:
        print('Label: Spam')
    else:
        print('Equal proabilities, have a human classify this!')

In [46]:
classify('WINNER!! This is the secret code to unlock the money: C3421.')

P(Spam|message): 1.3481290211300841e-25
P(Ham|message): 1.9368049028589875e-27
Label: Spam


In [38]:
classify("Sounds good, Tom, then see u there")

P(Spam|message): 4.651925051800531e-22
P(Ham|message): 8.695240383498824e-15
Label: Ham


## Measuring The Filter Spam Accuracy

The two results above look promising, but let's see how well the filter does on our test set, which has 1,114 messages.

We'll start by writing a function that returns classification labels instead of printing them.

In [47]:
def classify_test_set(message):    
    '''
    message: a string
    '''
    
    message = re.sub('\W', ' ', message)
    message = message.lower().split()
    
    p_spam_given_message = p_spam
    p_ham_given_message = p_ham

    for word in message:
        if word in parameters_spam:
            p_spam_given_message *= parameters_spam[word]
            
        if word in parameters_ham:
            p_ham_given_message *= parameters_ham[word]
    
    if p_ham_given_message > p_spam_given_message:
        return 'ham'
    elif p_spam_given_message > p_ham_given_message:
        return 'spam'
    else:
        return 'needs human classification'

Now that we have a function that returns labels instead of printing them, we can use it to create a new column in our test set.

In [48]:
test_set['predicted'] = test_set['SMS'].apply(classify_test_set)
test_set.head()

,Label,SMS,predicted
0,ham,Later i guess. I needa do mcat study too.,ham
1,ham,But i haf enuff space got like 4 mb...,ham
2,spam,Had your mobile 10 mths? Update to latest Oran...,spam
3,ham,All sounds good. Fingers . Makes it difficult ...,ham
4,ham,"All done, all handed in. Don't know if mega sh...",ham


Now, we'll write a function to measure the accuracy of our spam filter to find out how well our spam filter does.

In [49]:
correct = 0
total = test_set['SMS'].shape[0]

for row in test_set.iterrows():
    row = row[1]
    if row['Label'] == row['predicted']:
        correct += 1
        
print('Correct: {}'.format(correct))
print('Incorrect: {}'.format(total - correct))
print('Accuracy: {}'.format(correct/total))

Correct: 1100
Incorrect: 14
Accuracy: 0.9874326750448833


The accuracy is close to 98.74%, which is really good. Our spam filter looked at 1,114 messages that it hasn't seen in training, and classified 1,100 correctly.